<center>

# **Gather indicator data**

</center>

Some indicatores where from PAHO sources </br>
https://opendata.paho.org/en/core-indicators/download-dataset</br>


Others from WHO API</br>
https://www.who.int/data/gho/info/gho-odata-api


In [1]:
import pandas as pd
from prettytable import PrettyTable
import matplotlib.pyplot as plt
from collections import Counter
import requests
import io
import os

In [23]:
#Mount your Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
os.listdir('/content/drive/MyDrive/')

['MA 1022 PROF JIMMY CALVO',
 'Banco de Material MA 1022',
 'Schritte International. 1 Kursbuch + Arbeitsbuch.pdf',
 'proyecto_opti_SVM_UCR.ipynb',
 'Proyecto Análisis Datos III',
 'Tareas y Proyecto ALN',
 'Miscelaneo',
 'Proba_Mate_Pura',
 'Análisis de series de tiempo (Alvaro Montenegro García) (z-lib.org).pdf',
 'Data_Analysis',
 'Londres2022',
 'AnalisisDatos1',
 'AnalisisDatos2',
 'Schritte international 3 Kursbuch + Arbeitsbuch by Silke Hilpert et al. (z-lib.org).pdf',
 'Schritte international 2. Deutsch als Fremdsprache  Kursbuch + Arbeitsbuch by Daniela Niebisch et al. (z-lib.org).pdf',
 'Tesis de Maestría-Jimmy Calvo Monge.pdf',
 'UCR_Courses',
 'General_Math',
 'Building Machine Learning Systems with Python-2nd Edition-2015.pdf',
 'dokumen.pub_designing-machine-learning-systems-an-iterative-process-for-production-ready-applications-1nbsped-1098107969-9781098107963.pdf',
 'Designing Machine Learning Systems with Python-2016.pdf',
 'CountryCodes_Americas.xlsx']

## Open country codes for the Americas and the Caribbean

In [29]:
# output_path = '/content/drive/MyDrive/2023_ODS_OPS/Data/'
output_path = '/content/drive/MyDrive/' # Jimmy
file_name   = 'CountryCodes_Americas.xlsx'
file_path = os.path.join(output_path, file_name)

In [30]:
countries = pd.read_excel(file_path)
#df = df.drop_duplicates(subset='Country Name')
countries.head()
#len(countries["Country Code"].unique())

,Name,Code
0,Antigua,ATG
1,Argentina,ARG
2,Bahamas,BHS
3,Barbados,BRB
4,Belize,BLZ


In [32]:
# countries_code = countries['Country Code'].tolist()
countries_code = countries['Code'].tolist() # Jimmy

## **Retrieving WHO indicator data**

Download an indicator by specifying the indicator code. This will return all associated data for that specific indicator.

Available dimenstions http://ghoapi.azureedge.net/api/WHOSIS_000001

In [33]:
def get_indicator_data(indicator_code):
    url = f"https://ghoapi.azureedge.net/api/{indicator_code}"
    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()["value"]

        # Create empty lists to store the data
        IDs = []
        IndicatorCodes = []
        Countries = []
        years  = []
        values = []
        sex    = []
        Date1  = []
        Date2  = []


        # Extract the data and populate the lists
        for item in data:
            Id = item["Id"]
            IndicatorCode = item["IndicatorCode"]
            year          = item["TimeDimensionValue"]
            value         = item["NumericValue"]
            Country       = item["SpatialDim"]
            Sexo          = item["Dim1"]
            Date11        = item["TimeDimensionBegin"]
            Date22        = item["TimeDimensionEnd"]

            IDs.append(Id)
            IndicatorCodes.append(IndicatorCode)
            Countries.append(Country)
            years.append(year)
            values.append(value)
            sex.append(Sexo)
            Date1.append(Date11)
            Date2.append(Date22)

        # Create the dataframe
        df = pd.DataFrame({"Id": IDs,"SEX": sex, "IndicatorCode": IndicatorCodes, "Year": years, "Date1": Date1,"Date2":Date2,"Value": values, "Country": Countries})

        return df

    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)



## **Crude suicide rates (per 100 000 population) (SDG 3.4.2)**

The age-standardized mortality rate is a weighted average of the age-specific mortality rates per 100 000 persons, where the weights are the proportions of persons in the corresponding age groups of the WHO standard population.
</br>
</br>

**Other related indicator**</br>
Age-standardized suicide rates (per 100 000 population) (MH_12): </br>
</br>
The age-standardized mortality rate is a weighted average of the age-specific mortality rates per 100 000 persons, where the weights are the proportions of persons in the corresponding age groups of the WHO standard population.

In [6]:
#https://www.who.int/data/gho/data/themes/mental-health/suicide-rates
suicide_index         = "SDGSUICIDE"

In [34]:
# Call the function with the URL
df = get_indicator_data(suicide_index)

## Filter countries in the Americas and the Caribbean
suicide_info = df[df['Country'].isin(countries_code)]

##Filtrar solo informacion para ambos sexos
suicide_info = suicide_info[suicide_info['SEX']=='BTSX']

suicide_info1 = suicide_info[['IndicatorCode','Year','Country','Value']]
# Adding the 'IndicatorName' column with the value "Suicide"
suicide_info1['IndicatorName'] = "Crude Suicide Rate"
suicide_info1.head(5)


<ipython-input-34-cc1ac921bd33>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suicide_info1['IndicatorName'] = "Crude Suicide Rate"


,IndicatorCode,Year,Country,Value,IndicatorName
12,SDGSUICIDE,2019,ARG,11.77,Crude Suicide Rate
18,SDGSUICIDE,2019,ATG,0.00,Crude Suicide Rate
51,SDGSUICIDE,2019,BHS,3.35,Crude Suicide Rate
60,SDGSUICIDE,2019,BLZ,5.36,Crude Suicide Rate
63,SDGSUICIDE,2019,BOL,6.73,Crude Suicide Rate


In [ ]:
suicide_info.to_excel('/content/drive/MyDrive/2023_ODS_OPS/SuicideRate.xlsx')

## **Selected WHO Indicators**


In [ ]:
#List of selected indicators
path_selected_indicators ='/content/drive/MyDrive/2023_ODS_OPS/OutputFiles/3.List_of_Selected_Suicide_Indicators.xlsx'
SheetName = '3.Suicide_Selected_Indicators'

selected_indicators = pd.read_excel(path_selected_indicators,sheet_name=SheetName )
selected_index_code = selected_indicators['IndicatorCode']


In [ ]:
#Indicator 0 is from WHO

index_code = 'FINPROTECTION_IMP_NPRELPL_POP' #selected_index_code[2]
df         = get_indicator_data(index_code)

## Filter countries in the Americas and the Caribbean
Americas_info = df[df['Country'].isin(countries_code)]
Americas_info.head(5)

,Id,SEX,IndicatorCode,Year,Date1,Date2,Value,Country
7,31741675,TOTL,FINPROTECTION_IMP_NPRELPL_POP,2017,2017-01-01T00:00:00+01:00,2017-12-31T00:00:00+01:00,0.71,SLV
8,31741676,TOTL,FINPROTECTION_IMP_NPRELPL_POP,2018,2018-01-01T00:00:00+01:00,2018-12-31T00:00:00+01:00,0.63,SLV
9,31741680,TOTL,FINPROTECTION_IMP_NPRELPL_POP,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,0.61,SLV
189,31742300,TOTL,FINPROTECTION_IMP_NPRELPL_POP,2016,2016-01-01T00:00:00+01:00,2016-12-31T00:00:00+01:00,1.07,SUR
214,31721240,RUR,FINPROTECTION_IMP_NPRELPL_POP,1999,1999-01-01T00:00:00+01:00,1999-12-31T00:00:00+01:00,3.18,BOL


In [ ]:
##Filtrar solo informacion para ambos sexos
Americas_info = Americas_info[Americas_info['SEX']=='TOTL']
Americas_info1= Americas_info[['IndicatorCode','Year','Country','Value']]

# Adding the 'IndicatorName' column with the value "Suicide"
Americas_info1['IndicatorName'] = "Pop below a relative poverty line by household health expeditures"
Americas_info1.head(5)

<ipython-input-12-e8ed11e86fe9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Americas_info1['IndicatorName'] = "Pop below a relative poverty line by household health expeditures"


,IndicatorCode,Year,Country,Value,IndicatorName
7,FINPROTECTION_IMP_NPRELPL_POP,2017,SLV,0.71,Pop below a relative poverty line by household...
8,FINPROTECTION_IMP_NPRELPL_POP,2018,SLV,0.63,Pop below a relative poverty line by household...
9,FINPROTECTION_IMP_NPRELPL_POP,2019,SLV,0.61,Pop below a relative poverty line by household...
189,FINPROTECTION_IMP_NPRELPL_POP,2016,SUR,1.07,Pop below a relative poverty line by household...
215,FINPROTECTION_IMP_NPRELPL_POP,1999,BOL,1.49,Pop below a relative poverty line by household...


In [ ]:
# Get the unique values of the 'Year' column
unique_years = Americas_info['Year'].unique()

# Sort the unique years in ascending order
sorted_unique_years = sorted(unique_years)
print(sorted_unique_years)

['1990', '1991', '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']


## Identify the missing countries

In [ ]:
def find_unique_values(list1, list2):
  set1 = set(list1)
  set2 = set(list2)
  unique_values = list(set1.symmetric_difference(set2))
  return unique_values

unique_countries = Americas_info['Country'].unique()

result = find_unique_values(unique_countries, countries_code)
print('Total countries with data',len(unique_countries))

countries_name = countries[countries['Country Code'].isin(result)]
print(countries_name)


## PAHO Indicators

The excel with the whole information is dowloaded from the PAHO
https://opendata.paho.org/en/core-indicators/download-dataset</br>
</br>


In [ ]:
output_path = '/content/drive/MyDrive/2023_ODS_OPS/Data/'
file_name   = 'PAHO-Core-indicators-2023.csv'

file_path = os.path.join(output_path, file_name)

In [ ]:
#PAHO_DATA = pd.read_csv(file_path, error_bad_lines=False)
PAHO_DATA = pd.read_csv(file_path)
PAHO_DATA.head(5)

<ipython-input-14-008c0ce2a19f>:2: DtypeWarning: Columns (10,13,14,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  PAHO_DATA = pd.read_csv(file_path)


,paho_indicator_id,indicator_name,nombre_indicador,SpatialDimType,SpatialDim,SpatialDim_en,SpatialDim_es,TimeDimType,TimeDim,Numeric_value,...,data_provider_type,data_provider_specific,data_secondary_source,type_statistics,public_private,public_private_sp,source_url,preliminary,published_at,accessed_at
0,1017,Population aged < 15 years (thousands); female,Población < 15 años (en miles); mujer,COUNTRY,ABW,Aruba,Aruba,YEAR,1995,9.9125,...,Other,United Nations,PAHO Core Indicators,NaN,NaN,NaN,https://population.un.org/wpp/,0.0,11/7/2022,26/7/2022
1,1017,Population aged < 15 years (thousands); female,Población < 15 años (en miles); mujer,COUNTRY,ABW,Aruba,Aruba,YEAR,1996,9.8835,...,Other,United Nations,PAHO Core Indicators,NaN,NaN,NaN,https://population.un.org/wpp/,0.0,11/7/2022,26/7/2022
2,1017,Population aged < 15 years (thousands); female,Población < 15 años (en miles); mujer,COUNTRY,ABW,Aruba,Aruba,YEAR,1997,9.9220,...,Other,United Nations,PAHO Core Indicators,NaN,NaN,NaN,https://population.un.org/wpp/,0.0,11/7/2022,26/7/2022
3,1017,Population aged < 15 years (thousands); female,Población < 15 años (en miles); mujer,COUNTRY,ABW,Aruba,Aruba,YEAR,1998,10.0205,...,Other,United Nations,PAHO Core Indicators,NaN,NaN,NaN,https://population.un.org/wpp/,0.0,11/7/2022,26/7/2022
4,1017,Population aged < 15 years (thousands); female,Población < 15 años (en miles); mujer,COUNTRY,ABW,Aruba,Aruba,YEAR,1999,10.1590,...,Other,United Nations,PAHO Core Indicators,NaN,NaN,NaN,https://population.un.org/wpp/,0.0,11/7/2022,26/7/2022


In [ ]:
filtered_df = PAHO_DATA[PAHO_DATA['SpatialDim'].isin(countries_code)]
final_df    = filtered_df[['paho_indicator_id',	'indicator_name', 'SpatialDim',	'SpatialDim_en', 'TimeDim','Numeric_value']]

## Explore particular indicators

In [ ]:
#List of selected indicators
path_selected_indicators ='/content/drive/MyDrive/2023_ODS_OPS/OutputFiles/3.List_of_Selected_Suicide_Indicators.xlsx'
SheetName = '3.Suicide_Selected_Indicators'

selected_indicators = pd.read_excel(path_selected_indicators,sheet_name=SheetName )
paho_selected_index_code = selected_indicators['IndicatorCode']


In [ ]:
index_code = 1304 #paho_selected_index_code[281]
PAHO_info  = final_df[ final_df['paho_indicator_id']==index_code]
sorted(PAHO_info['TimeDim'].unique())

[2000, 2005, 2010, 2015, 2018, 2019, 2020, 2023, 2025]

### Create a dataframe with all years and missing information

In [ ]:
#indicator from 3 to 31 are from PAHO

# Define an empty list to store the results for each indicator
indicator_results = []


for i in range(3,43):
  #Filter particular indicator
  index_code = selected_index_code[i]
  PAHO_info  = final_df[final_df['paho_indicator_id']==index_code]

  #Fing years availables
  unique_years = PAHO_info['TimeDim'].unique()
  sorted_unique_years = sorted(unique_years)
  first_year = sorted_unique_years[0]
  last_year  = sorted_unique_years[-1]
  Years = f"{first_year}-{last_year}"

  if len(unique_years)>1:
    #Find frequency or periods
    differences = [unique_years[i+1] - unique_years[i] for i in range(len(unique_years) - 1)]
    frequency_count = Counter(differences)
    most_common_difference = frequency_count.most_common(1)
    frequency = most_common_difference[0][0]
  else:
    frequency = 0


  ##identify how many countries have data
  def find_unique_values(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    unique_values = list(set1.symmetric_difference(set2))
    return unique_values

  unique_countries = PAHO_info['SpatialDim'].unique()
  Total_countries = len(unique_countries)

  ##Names of missing Countries
  missing =  find_unique_values(unique_countries, countries_code)
  missing_countries = countries[countries['Country Code'].isin(missing)]
  missing_countries_names = missing_countries['Country Name'].tolist()

  #Identify missing values
  total_empty_rows = PAHO_info['Numeric_value'].isna().sum()

  # Append the results for the current indicator to the list
  indicator_results.append({
        'index': index_code,
        'Missing countries': missing_countries_names,
        'Countries with Data': Total_countries,
        'Periodicidad Years available': frequency,
        'Years available': Years,
        'Missing data': total_empty_rows
    })

# Create a DataFrame from the list of indicator results
df = pd.DataFrame(indicator_results)
df.head(5)

,index,Missing countries,Countries with Data,Periodicidad Years available,Years available,Missing data
0,304,"[Dominica, St. Kitts and Nevis]",33,1,2000-2019,0
1,305,"[Dominica, St. Kitts and Nevis]",33,1,2000-2019,0
2,306,"[Dominica, St. Kitts and Nevis]",33,1,2000-2019,0
3,281,"[Antigua and Barbuda, Bahamas, The, Barbados, ...",20,2,1995-2021,0
4,80,"[Antigua and Barbuda, Bahamas, The, Barbados, ...",25,1,1995-2021,0


In [ ]:
df.to_excel('/content/drive/MyDrive/2023_ODS_OPS/missingValues.xlsx')

## Filter indicators of interest from PAHO Data

In [ ]:
paho_final_indicators = paho_selected_index_code[1:41]

PAHO_Values = PAHO_DATA[PAHO_DATA['paho_indicator_id'].isin(paho_final_indicators)]
PAHO_Values1 = PAHO_Values[['paho_indicator_id','TimeDim','SpatialDim','Numeric_value','indicator_name']]

# Define a dictionary for mapping old column names to new column names
new_column_names = {
    'paho_indicator_id': 'IndicatorCode',
    'TimeDim': 'Year',
    'SpatialDim': 'Country',
    'Numeric_value': 'Value',
    'indicator_name': 'IndicatorName'
}

# Use the 'rename' method to change column names on the copied DataFrame
PAHO_Values1.rename(columns=new_column_names, inplace=True)

PAHO_Values1.head(5)

<ipython-input-19-c99733f77a88>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PAHO_Values1.rename(columns=new_column_names, inplace=True)


,IndicatorCode,Year,Country,Value,IndicatorName
72,1016,1995,ABW,20.0400,Population aged < 15 years (thousands)
73,1016,1996,ABW,19.9725,Population aged < 15 years (thousands)
74,1016,1997,ABW,20.0385,Population aged < 15 years (thousands)
75,1016,1998,ABW,20.2270,Population aged < 15 years (thousands)
76,1016,1999,ABW,20.5080,Population aged < 15 years (thousands)


In [ ]:
# Concatenate the DataFrames vertically
combined_df = pd.concat([Americas_info1, PAHO_Values1], ignore_index=True)

print(combined_df.shape)
combined_df.head(5)

(31456, 5)


,IndicatorCode,Year,Country,Value,IndicatorName
0,FINPROTECTION_IMP_NPRELPL_POP,2017,SLV,0.71,Pop below a relative poverty line by household...
1,FINPROTECTION_IMP_NPRELPL_POP,2018,SLV,0.63,Pop below a relative poverty line by household...
2,FINPROTECTION_IMP_NPRELPL_POP,2019,SLV,0.61,Pop below a relative poverty line by household...
3,FINPROTECTION_IMP_NPRELPL_POP,2016,SUR,1.07,Pop below a relative poverty line by household...
4,FINPROTECTION_IMP_NPRELPL_POP,1999,BOL,1.49,Pop below a relative poverty line by household...


In [ ]:
combined_df.to_excel('/content/drive/MyDrive/2023_ODS_OPS/OutputFiles/4.Values_Suicide_rate_Indicators_PAHO.xlsx')

# The world Bank

There some additional data dowloaded from
https://databank.worldbank.org/source/world-development-indicators/preview/on#

## Children-Adolescent data from WHO Indicators with 'children/adolescent' keys
08/23/2023
Jimmy


In [39]:
# Children-Adolescent data
# 08/23/2023
# Jimmy

index_codes = ['CC_7', 'CC_8', 'AIR_8', 'CC_5',
               'CC_6', 'AIR_4', 'AIR_10', 'AIR_12', 'AIR_16', 'AIR_6',
               'RSUD_10', 'RSUD_340', 'RSUD_350', 'RSUD_890', 'RSUD_880',
               'RSUD_900', 'TOTENV_11', 'TOTENV_8', 'TOTENV_10', 'TOTENV_12',
               'TOTENV_7', 'TOTENV_9', 'AIR_71', 'NCD_CCS_PA_GUIDELINES_5to19',
               'NCD_CCS_PA_GUIDELINES_UNDER5', 'AIR_72', 'AIR_75', 'AIR_73',
               'CHILDVIOL', 'AIR_74']
children_ad_data = []
for index_code in index_codes:
  print(f'Processing {index_code} ...')
  df = get_indicator_data(index_code)
  ## Filter countries in the Americas and the Caribbean
  df = df[df['Country'].isin(countries_code)]
  children_ad_data.append(df)
children_ad_data = pd.concat(children_ad_data, ignore_index=True)
children_ad_data

Processing CC_7 ...
Processing CC_8 ...
Processing AIR_8 ...
Processing CC_5 ...
Processing CC_6 ...
Processing AIR_4 ...
Processing AIR_10 ...
Processing AIR_12 ...
Processing AIR_16 ...
Processing AIR_6 ...
Processing RSUD_10 ...
Processing RSUD_340 ...
Processing RSUD_350 ...
Processing RSUD_890 ...
Processing RSUD_880 ...
Processing RSUD_900 ...
Processing TOTENV_11 ...
Processing TOTENV_8 ...
Processing TOTENV_10 ...
Processing TOTENV_12 ...
Processing TOTENV_7 ...
Processing TOTENV_9 ...
Processing AIR_71 ...
Processing NCD_CCS_PA_GUIDELINES_5to19 ...
Processing NCD_CCS_PA_GUIDELINES_UNDER5 ...
Processing AIR_72 ...
Processing AIR_75 ...
Processing AIR_73 ...
Processing CHILDVIOL ...
Processing AIR_74 ...


,Id,SEX,IndicatorCode,Year,Date1,Date2,Value,Country
0,29491145,BTSX,AIR_8,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,3137.85099,ARG
1,29491146,FMLE,AIR_8,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,1344.60255,ARG
2,29491147,MLE,AIR_8,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,1793.24844,ARG
3,29491151,BTSX,AIR_8,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,5.23194,ATG
4,29491152,FMLE,AIR_8,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,2.40117,ATG
...,...,...,...,...,...,...,...,...
1670,29527521,FMLE,AIR_74,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,0.10522,VCT
1671,29527522,MLE,AIR_74,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,0.12945,VCT
1672,29527523,BTSX,AIR_74,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,60.43387,VEN
1673,29527524,FMLE,AIR_74,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00,26.39030,VEN


In [42]:
indicator_descs_df = pd.DataFrame({
    "IndicatorDesc": [
        "Climate change attributable DALYs ('000) in children under 5 years",
        "Climate change  attributable DALYs per 100'000 children under 5 years",
        "Ambient air pollution attributable DALYs  in children under 5 years",
        "Climate change attributable deaths ('000) in children under 5 years",
        "Climate change  attributable deaths per 100'000 children under 5 years",
        "Ambient air pollution attributable deaths in children under 5 years",
        "Ambient air pollution  attributable DALYs per 100'000 children under 5 years",
        "Household air pollution attributable deaths in children under 5 years",
        "Household air pollution attributable DALYs in children under 5 years",
        "Ambient air pollution  attributable deaths per 100'000 children under 5 years",
        "National survey on substance use among children and adolescents",
        "Treatment programmes for children and adolescents with alcohol use disorders",
        "Treatment programmes for children and adolescents with drug use disorders",
        "Treatment programmes for children and adolescents with alcohol use disorders",
        "Epidemiological data collection system for substance use among children and adolescents",
        "Treatment programmes for children and adolescents with drug use disorders",
        "Deaths attributable to the environment in children under 5 years (%)",
        "Total environment  attributable deaths per 100'000 children under 5 years",
        "Total environment  attributable DALYs per 100'000 children under 5 years",
        "DALYs attributable to the environment in children under 5 years (%)",
        "Total environment attributable deaths ('000) in children under 5 years",
        "Total environment attributable DALYs ('000) in children under 5 years",
        "Ambient air pollution attributable DALYs in children under 15 years",
        "Existence of national guidelines for physical activity for children and adolescents aged 5-19",
        "Existence of national guidelines for physical activity for children under 5",
        "Ambient air pollution attributable deaths in children under 15 years",
        "Household air pollution attributable DALYs in children under 15 years",
        "Ambient air pollution attributable YLL in children under 15 years",
        "Indicator 16.2.1: Proportion of children aged 1-17 years who experienced any physical punishment and/or psychological aggression by caregivers in the past month",
        "Household air pollution attributable deaths in children under 15 years"],
    "IndicatorCode": index_codes
    })
indicator_descs_df

,IndicatorDesc,IndicatorCode
0,Climate change attributable DALYs ('000) in ch...,CC_7
1,Climate change attributable DALYs per 100'000...,CC_8
2,Ambient air pollution attributable DALYs in c...,AIR_8
3,Climate change attributable deaths ('000) in c...,CC_5
4,Climate change attributable deaths per 100'00...,CC_6
5,Ambient air pollution attributable deaths in c...,AIR_4
6,Ambient air pollution attributable DALYs per ...,AIR_10
7,Household air pollution attributable deaths in...,AIR_12
8,Household air pollution attributable DALYs in ...,AIR_16
9,Ambient air pollution attributable deaths per...,AIR_6


In [43]:
children_ad_data = children_ad_data.merge(indicator_descs_df, left_on='IndicatorCode', right_on='IndicatorCode', how='left')
children_ad_data.to_excel('/content/drive/MyDrive/WHO_children_adolescent_data.xlsx')